Código para conectarse a PubMed (NCBI) mediante el sistema Entrez. 

Input: Dataset Especies
Output: Dataset Publicaciones Científicas

Se utilizan los nombres de las especies presentes en los predios como campo de búsqueda para extraer publicaciones científicas

In [1]:
#Importo librerías
from openpyxl import load_workbook
import pandas as pd
import json
from Bio import Entrez

In [ ]:
#Dataset predi
df_especies=pd.read_excel('Terrenos.xlsx',sheet_name='Hoja1')
columna=df_especies['Nombre científico']

In [ ]:
#Adición del código de la especie al json
especies_dict=dict(zip(df_especies['Nombre científico'],df_especies['ID']))


In [ ]:
#Primero acceder a esearch: se obtiene una lista de ids de papers

Entrez.email = 'mbugaldej@gmail.com'
topics = columna 

data=[]

for topic in topics:
    
    handle = Entrez.esearch(db='pubmed', retmax=2, term=topic)
    record = Entrez.read(handle)
    id_list = record['IdList']     #record.get('IdList',[]) lista vacía si no hay resultados
    print(topic,": ",len(id_list)) #revisión interna
    print(id_list)                 #revisión interna

    for pmid in id_list:
        #try:
            handle= Entrez.efetch(db='pubmed', retmax=2, id=pmid, retmode='xml')
            records = Entrez.read(handle)
            
       
            for record in records.get('PubmedArticle',[]):
                    #print(json.dumps(record, indent=4, default=str))
                #try:

                    medline = record['MedlineCitation']['Article']
                    title = medline.get('ArticleTitle','null')
                    abstract=''.join(medline['Abstract']['AbstractText']) if 'Abstract' in medline else ''
                    #language= medline.get('Language','null')[0]
                    url=f"https://www.ncbi.nlm.nih.gov/pubmed/{pmid}"
                    journal=''.join(medline['Journal']['Title']) if 'Journal' in medline else ''
                    keywords= []
                    if 'KeywordList' in record['MedlineCitation'] and record['MedlineCitation']['KeywordList']:
                           keywords = [kw for kw in record['MedlineCitation']['KeywordList'][0]]
                    else:
                           keywords=[]

                    #ID especie:
                    #topic_id=''.join(e for e in df_especies['ID'] if df_especies['Nombre científico']==topic)
                    #topic_id=df_especies.loc[df_especies['Nombre científico']==topic,'ID'].values[0]
                    topic_id=especies_dict[topic]

                    date=record['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']
                    year=date.get('Year','')
                    month=date.get('Month','')
                    day=date.get('Day','')
                    #authors and affiliations
                    authors_list=[]
                    if 'AuthorList' in medline:
                           for author in medline['AuthorList']:
                                  lastname=f"{author.get('LastName',' ')}"
                                  name=f"{author.get('ForeName','')}"
                                  affiliations=[]
                                  if 'AffiliationInfo' in author:
                                         affiliations=[aff['Affiliation']for aff in author['AffiliationInfo']if'Affiliation'in aff]
                                  else:
                                         ""
                                  authors_list.append({
                                         "Name":name,
                                         "Lastname":lastname,
                                         "Affiliations":affiliations
                                  })    
                    



                    #Generar el JSON:
                    data.append({
                        "PMID":pmid,
                        "Topic":topic,
                        "ID":topic_id,
                        "Date":{
                               "Day":day,
                               "Month":month,
                               "Year":year,                                                            
                        },
                        "Authors":authors_list,
                        "Journal":journal,
                        "Keywords":keywords,
                        "Title":title,
                        "Abstract":abstract,
                        "URL":url
                     })

pubmed=json.dumps(data, indent=4,ensure_ascii=False)

Abies alba :  2
['40005675', '39896777']
Acacia dealbata :  2
['39437840', '39011657']
Alnus glutinosa :  2
['39892071', '39822710']
Amomyrtus meli :  1
['33692648']
Aextoxicon punctatum :  2
['32107298', '28065922']
Acacia melanoxylon :  2
['38732191', '38664272']
Cupresus sp :  2
['30022505', '18654914']
Chusquea quila :  2
['37496974', '24963379']
Cupressus sempervirens :  2
['39975607', '39875714']
Cupressus lusitanica :  2
['39286326', '38973876']
Drimis wintei :  2
['36258653', '35062305']
Eucriphia cordifolia :  2
['39974656', '39962423']
Eucalyptus globulus :  2
['40008063', '40007963']
Gevuina avellana :  2
['38721332', '38364303']
Lomatia hirsuta :  2
['33679850', '29997642']
Juncus sp :  2
['39830292', '39140100']
Myrceugenia exsucca :  2
['37418157', '33692648']
Aristotelia chilensis :  2
['39861548', '39728122']
Nothofagus alpina :  2
['38407127', '33423354']
Nothofagus dombeyi :  2
['39167704', '38279025']
Eucalyptus nitens :  2
['39739612', '39125241']
Nothofagus obliqua

In [26]:
pubmed=json.dumps(data, indent=4,ensure_ascii=False)

In [ ]:
#Genero el archivo para analizar posteriormente
archivo_json="pubmed_data262.json"             

with open(archivo_json,"w",encoding="utf-8") as f:
    f.write(pubmed)
print("JSON generado")

JSON generado


In [ ]:
#chequeo interno
pubmed_json[0]

{'PMID': '39861548',
 'Topic': 'Aristotelia chilensis',
 'Date': {'Day': '12', 'Month': 'Jan', 'Year': '2025'},
 'Authors': [{'Name': 'Fiorella',
   'Lastname': 'Calderón-Ureña',
   'Affiliations': ['Departamento de Silvicultura, Facultad de Ciencias Forestales, Universidad de Concepción, Concepción 4070374, Chile.',
    'Centro Tecnológico de la Planta Forestal, Instituto Forestal, Sede Biobío, San Pedro de la Paz 4130946, Chile.']},
  {'Name': 'Carolina',
   'Lastname': 'Álvarez-Maldini',
   'Affiliations': ['Departamento de Silvicultura, Facultad de Ciencias Forestales, Universidad de Concepción, Concepción 4070374, Chile.']},
  {'Name': 'Manuel',
   'Lastname': 'Acevedo',
   'Affiliations': ['Centro Tecnológico de la Planta Forestal, Instituto Forestal, Sede Biobío, San Pedro de la Paz 4130946, Chile.']},
  {'Name': 'Manuel E',
   'Lastname': 'Sánchez-Olate',
   'Affiliations': ['Departamento de Silvicultura, Facultad de Ciencias Forestales, Universidad de Concepción, Concepción 40

In [ ]:
#CON CITA
#Primero acceder a esearch: se obtiene una lista de ids de papers

Entrez.email = 'mbugaldej@gmail.com'
topics = ['Aristotelia chilensis']  # prueba
#topics = columna.tolist() #real: especies presentes en predios

data=[]

for topic in topics:
    
    handle = Entrez.esearch(db='pubmed', retmax=2, term=topic)
    record = Entrez.read(handle)
    id_list = record['IdList']     #record.get('IdList',[]) lista vacía si no hay resultados
    print(topic,": ",len(id_list)) #revisión interna
    print(id_list)                 #revisión interna

    for pmid in id_list:
        #try:
            handle= Entrez.efetch(db='pubmed', retmax=2, id=pmid, retmode='xml')
            records = Entrez.read(handle)

            for record in records.get('PubmedArticle',[]):
                #try:
                    medline = record['MedlineCitation']['Article']
                    title = medline.get('ArticleTitle','null')
                    abstract=''.join(medline['Abstract']['AbstractText']) if 'Abstract' in medline else ''
                    language= medline.get('Language','null')[0]
                    url=f"https://www.ncbi.nlm.nih.gov/pubmed/{pmid}"
                    journal=''.join(medline['Journal']['Title']) if 'Journal' in medline else ''
                    keywords= []
                    if 'KeywordList' in record['MedlineCitation'] and record['MedlineCitation']['KeywordList']:
                           keywords = [kw for kw in record['MedlineCitation']['KeywordList'][0]]
                    else:
                           keywords=[]
                    date=record['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']
                    year=date.get('Year','')
                    month=date.get('Month','')
                    day=date.get('Day','')
                    #authors and affiliations
                    authors_list=[]
                    if 'AuthorList' in medline:
                           for author in medline['AuthorList']:
                                  lastname=f"{author.get('LastName',' ')}"
                                  name=f"{author.get('ForeName','')}"
                                  affiliations=[]
                                  if 'AffiliationInfo' in author:
                                         affiliations=[aff['Affiliation']for aff in author['AffiliationInfo']if'Affiliation'in aff]
                                  else:
                                         ""
                                  authors_list.append({
                                         "Name":name,
                                         "Lastname":lastname,
                                         "Affiliations":affiliations
                                  })    
                     #Referencias:
                    if 'PubmedData' in record and 'ReferenceList' in record['PubmedData']:
                          references = []
                          for ref_list in record['PubmedData']['ReferenceList']:
                                if 'ReferenceList' in ref_list:
                                      for ref in ref_list['Reference']:
                                            citation=ref.get("Citation")
                                            references.append(citation)
                    else:
                          references=[]
                     
                    #ID especie:
                    topic_id=df_especies.loc[df_especies['Nombre científico']==topic,'ID'].values[0]



                    #Generar el JSON:
                    data.append({
                        "PMID":pmid,
                        "Topic":topic,
                        "ID":topic_id,
                        "Date":{
                               "Day":day,
                               "Month":month,
                               "Year":year,                                                            
                        },
                        "Authors":authors_list,
                        "Journal":journal,
                        "Keywords":keywords,
                        "Title":title,
                        "Abstract":abstract,
                        "URL":url,
                        "References":references
                     })

                    pubmed=json.dumps(data, indent=4,ensure_ascii=False)